# Procena popularnosti video igre GTA VI primenom višestruke linearne regresije 

## Uvod

Cilj ovog projekta jeste napraviti model koji će uspešno da opisuje broj igrača jedne igrice razmatrajući razne faktore. Uspešno predviđanje buduće igranosti ove visoko iščekivane igre može dovesti do daljih korisnih zaključaka kao što je zarada kompanije. 
Projekat je generalizovao procenu popularnosti na bilo koju igru, dok je GTA 6 bio cilj zbog iščekivanosti njegovog izlaska. 

Kroz GUI programa je omogućeno korisnicima da unesu podatke za neku igricu i potom se GUI ispisuje procenu broja igrača koju je ostvario model. GUI je rađen kroz PyQt biblioteku. 

## Skup podataka

#### Prvi skup podataka

Prvi korak u izradi ovog projekta jeste razmatranje koji sve faktori mogu da utiču na broj igrača. Primer jednog uzorka, tj. faktora jedne igre jeste:

In [ ]:
{
  "name": "The Elder Scrolls V: Skyrim", # Name korišćen samo radi preglednosti
  "daily_players": 25000, # Broj igrača u trenutku izrade projekta (ciljana vrednost)
  "rating": 0.95, # Ocena igrice (intuitivno ukoliko je ocena veća, broj igrača je veći)
  "company_budget": 80000000, # Budžet kompanije za izradu igrice (intuitivno ukoliko je budžet veći, broj igrača je veći)
  "years_released": 4, # Koliko davno je igrica puštena u javnost (intuitivno što dalje, broj igrača manji)
  "trailer_views": 15000000, # Broj prikaza na trejleru (intuitivno ukoliko podržava broj igrača je veći)
  "google_trend": 0.4, # Google trend jeste merilo aktuelnosti neke teme na internetu (što veći trend, veći broj igrača)
  "game_of_the_year": true, # Nagrada igra godine (Iako se za buduću igru ne zna da li će dobiti nagradu, može se pretpostaviti)
  "multiplayer": true, # Da li podržava multiplayer (intuitivno ukoliko podržava broj igrača je veći)
  "platform_availability": { # Dostupnost na platformama (intuitivno ako je dostupna na više platforma, broj igrača je veći)
    "PC": true,
    "PLAYSTATION": true,
    "XBOX": true,
    "PHONE": false
  }, 
  "genre": { # Žanr igrice korišćen za razlikovanje uspešnosti po žanru
    "action": false,
    "adventure": true,
    "rpg": true,
    "simulation": false,
    "sports": false,
    "puzzle": false,
    "horror": false,
    "survival": false,
    "indie": false,
    "fps": false,
    "mmo": true,
    "open_world": true
  }
}

Gore prikazani dataset je prvi pokušaj pravljenja validnog dataseta. Mada kroz testiranje dataset-a (od 60-tak igrica) nad raznim modelima, nikakva linearnost se nije mogla videti. Kao glavni test model bio je OLS, koji je padao na skoro sve linearne pretpostavke.


Sledeći korak jeste razmatranje zašto je ovaj dataset "loš" i njegova prepravka. Odlučio sam da obrišem game_of_the_year, rating budući da se oni nisu mogli znati unapred. Pored brisanja 2 stavke, model se nije značajno promenio. Glavni tok misli za ovaj model jeste da daily players najviše prati koliko je davno puštena igrica, ubaci budžet kompanije, broj prikaza i google trend. Međutim, model nije uspeo da uspešno da prepozna šablon pa nije bio ni validan za ovaj projekat. 

Ono što je dodatno doprinelo njegovom neuspehu jeste nedostatak informacija za budžet kompanije koji je bio vrlo važan podatak.
Dopunjavanje ovog podatka je nerealno prikazivao budžet kompanije za igricu.

#### Drugi skup podataka

Nakon neuspešnog prvog dataset-a, shvatio sam da treba naći jedan ili dva faktora koji najviše utiču na model i ne tražiti previše od modela. Odlučio sam da promenim ciljanu vrednost (y) da bude broj igrača u momentu kada je izašla igrica. 

Zbog ove odluke, google trend danas i years_released nisu imali toliko veliki značaj jer se procenjivalo u trenutku izlaska igrice.
Statistika za broj igrača se mogla naći samo na Steam-u (platformi za video igre), pa je ceo fokus bio da se nađe broj igrača pri izlasku na Steam-u. 

Renovirani skup podataka izgleda ovako: 

In [ ]:
{
  "name": "The Elder Scrolls V: Skyrim",
  "players_on_launch": 69000, # ciljana vrednost za predviđanje
  "players_after_1year": 18000, # ili ova
  "year_of_release": 2016, # Faktor koji želi da pomogne modelu da shvati Kasnije => više igrača ukupno => više igrača u igri
  "company_budget": 80000000, # Glavni faktor na koga se oslanja model
  "trailer_views": 15000000, # Glavni faktor na koga se oslanja model
  "multiplayer": true, 
  "platform_availability": {
    "PC": true,
    "PLAYSTATION": true,
    "XBOX": true
  },
  "genre": {
    "action": false,
    "adventure": true,
    "rpg": true,
    "simulation": false,
    "sports": false,
    "puzzle": false,
    "horror": false,
    "survival": false,
    "indie": false,
    "fps": false,
    "mmo": true,
    "open_world": true,
    "story_mode": false,
    "strategy": false
  }
}

Renovirani skup podataka direktnije opisuje broj igrača i jednostavnije za razumevanje, međutim na račun ograničenja samo na Steam dostupne igre. Zbog ovoga, sve video igre jesu PC igre, pa ih možemo izbaciti.

## Metodologija

Koraci koji su potrebni da bi se dobro napravio model su sledeći:

- Učitavanje skupa podataka i njegova regularizacija
- Biranje metrike RMSE i R-squared
- Biranje modela i njegova implementacija 
- Procena modela i prepravke

## Implementacija

### Dataset

Predprocesiraćemo podatke iz dataset-a. Pošto je dataset u JSON formatu i imamo true/false vrednosti, prvo ćemo ukloniti ugneždene faktore i pretvoriti bool vrednosti u 0/1. 

In [ ]:
with open('data/dataset.json', 'r') as file:
    data = json.load(file)
games_list = data['games']
flattened_data = pd.json_normalize(games_list)
dataset = pd.DataFrame(flattened_data).drop('name', axis=1)
for col in TO_CONVERT:
    dataset[col] = dataset[col].astype(int)
X = dataset.drop(TO_DROP, axis=1) # TO_DROP predstavlja faktore koji nisu ciljna vrednost i faktore koje nećemo koristiti pri izradi modela
y = dataset[y_var] # predstavlja ciljanu vrednost

Sledeći korak jeste podela dataset-a na train/val podatke. 

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)

### Metrika

Za procenjivanje uspešnosti modela koristićemo Root Mean Squared Error iz biblioteke numpy (RMSE se koristi za procenu prosečne veličine greške između stvarnih i predviđenih vrednosti ciljne promenljive).

In [ ]:
import numpy as np
rmse = np.sqrt(mean_squared_error(y_val, train_predictions))
print("RMSE on validation set:", rmse)

Pored RMSE, koristićemo R-squared kako bi tačnije procenili model.

### Modeli

#### Huber regresioni model

Huber regresija je tehnika robustne regresije koja je manje osetljiva na ekstremne vrednosti u podacima u poređenju sa običnom linearnom regresijom (OLS). Korisna je posebno kada skup podataka sadrži ekstremne vrednosti ili greške koje ne prate normalnu raspodelu.

U Huber regresiji, funkcija gubitka koja se koristi za optimizaciju je kombinacija kvadratne greške (kao kod OLS regresije) za male greške i apsolutne greške za velike greške. Ovo čini model manje podložnim na ekstremne vrednosti jer se uticaj velikih grešaka smanjuje u poređenju sa OLS regresijom.

Huber regresioni model je testiran, međutim nije se proslavio. Naš dataset je od početka bio potpun jer se uzimala statistika sa Steam-a i nije bilo potrebna veštačka dopuna, pa su outlajeri što se tiče ovog dela odsutni. Medjutim, mogući su outlajeri što se tiče abnormalnih brojeva u datasetu. Sa tim da je dataset uzet nad 63 steam igrice koje su dosta raznovrsne, prepoznavanje tačnih outlajera nad malim datasetom sa toliko mnogo faktora je skoro nemoguće. Zbog ovog razloga nije korišćen Huber regresioni model.

#### ElasticNet model

Elastic Net je tehnika regularizovane regresije koja kombinuje L1 (LASSO) i L2 (Ridge) regularizaciju kako bi se smanjio overfitting i poboljšala generalizacija modela. Ova tehnika je posebno korisna kada postoji korelacija između faktora u podacima, jer LASSO i Ridge regularizacija imaju različite efekte na koeficijente regresije.

Sa velikim skupom podataka, postoji veća verovatnoća da će se susresti irelevantne karakteristike. Mogućnost ElasticNet-a da primenjuje i L1 (Lasso) i L2 (Ridge) pomaže u rešavanju prenaučenosti tako što smanjuje koeficijente i pojednostavljuje model. Ova regularizacija je posebno korisna kada postoji mnogo karakteristika, od kojih neke možda nisu relevantne za ciljnu promenljivu. 

Uzmimo u obzir da je naš dataset relativno mali u odnosu na broj faktora. Sa pretpostavkom da ElasticNet neće mnogo pomoći pri pravljenju validnog modela i dalje je testiran. 

In [ ]:
# SVI FAKTORI
# R-squared (ElasticNet): 0.2103721092648091
# Adjusted R-squared: 0.09192792565453056
# RMSE on validation set: 91991.86115539471
# 
# BRISANJE ŽANR FAKTORA
# R-squared (ElasticNet): 0.3772549674651906
# Adjusted R-squared: -0.10177967294620127
# RMSE on validation set: 89753.57933641819

Iako je RMSE relativno dobar, R-kvadrat je loš, što može ukazivati na nedostatak reprezentativnosti ili obuhvata u skupu podataka. 

#### RANSAC model

Glavna ideja RANSAC algoritma je da iterativno procenjuje parametre modela na osnovu nasumično izabranog podskupa podataka (tzv. "inliers"), a zatim procenjene parametre koristi za identifikaciju drugih podataka koji se dobro podudaraju sa modelom. Podaci koji se ne podudaraju sa modelom smatraju se ekstremnim vrednostima i izostavljaju se iz procene parametara u narednim iteracijama.
Ovo je nepogodno za ovaj dataset i ovaj problem jer ima dosta faktora i raznovrsnih igrica, tako da ako se uzme
delić dataset-a, moze značajno da promeni model.

#### RandomForest model

Vrlo slična situacija kao i kod RANSAC modela, RandomForest uzima deliće dataseta i pravi model nad njima i pravi predikcije,
potom kombinuje te predikcije. Ovo je nemoguće na ovom datasetu jer je vrlo mali i uzimanjem delića dataset-a mogu se uzeti
znatno različiti podskupovi, pa ih je nemoguce kombinovati validno.

#### OLS model

Budući da je OLS 'najjednostavniji model' gde ukoliko 1 ili 2 faktora imaju najveći uticaj na ciljnu promenljivu njegovo korišćenje se može isplatiti. Kod nas je upravo to situacija. Polja 'trailer_views' i 'company_budget' imaju najveći uticaj na 'players_on_lauch'.

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
model_statsmodel = sm.OLS(y_train, X_train)
results = model_statsmodel.fit()
print(results.summary())

In [ ]:
# =====================================================================================================
#                                         coef    std err          t      P>|t|      [0.025      0.975]
# -----------------------------------------------------------------------------------------------------
# year_of_release                      38.0871     46.565      0.818      0.420     -57.011     133.185
# company_budget                       -0.0005      0.000     -1.864      0.072      -0.001    5.24e-05
# trailer_views                         0.0027      0.002      1.637      0.112      -0.001       0.006
# multiplayer                        4.772e+04   5.54e+04      0.861      0.396   -6.55e+04    1.61e+05
# platform_availability.PLAYSTATION -1.382e+05   9.17e+04     -1.508      0.142   -3.25e+05     4.9e+04
# platform_availability.XBOX         1.212e+05   8.05e+04      1.505      0.143   -4.32e+04    2.86e+05
# genre.action                       8.362e+04   5.79e+04      1.445      0.159   -3.46e+04    2.02e+05
# genre.adventure                   -1.765e+05   6.22e+04     -2.837      0.008   -3.03e+05   -4.94e+04
# genre.rpg                          6.741e+04   7.15e+04      0.943      0.353   -7.86e+04    2.13e+05
# genre.simulation                  -8845.6416   7.48e+04     -0.118      0.907   -1.62e+05    1.44e+05
# genre.sports                      -4.627e+04   1.24e+05     -0.372      0.712      -3e+05    2.08e+05
# genre.puzzle                       9.181e+04   1.25e+05      0.736      0.468   -1.63e+05    3.47e+05
# genre.horror                      -4.695e+04   7.32e+04     -0.642      0.526   -1.96e+05    1.02e+05
# genre.survival                    -3637.2169      1e+05     -0.036      0.971   -2.09e+05    2.01e+05
# genre.indie                       -3378.0211   6.75e+04     -0.050      0.960   -1.41e+05    1.35e+05
# genre.fps                          1.101e+04   1.01e+05      0.109      0.914   -1.95e+05    2.17e+05
# genre.mmo                         -3.531e+04    1.1e+05     -0.321      0.751    -2.6e+05    1.89e+05
# genre.open_world                   1.453e+05   9.25e+04      1.570      0.127   -4.36e+04    3.34e+05
# genre.story_mode                    5.89e+04   1.08e+05      0.547      0.589   -1.61e+05    2.79e+05
# genre.strategy                    -4.253e+04   7.81e+04     -0.545      0.590   -2.02e+05    1.17e+05
# =====================================================================================================

Na summary(), možemo uočiti da faktori poput survival, indie, fps, survival, simulation ne utiču mnogo na model pa ih možda možemo ukloniti. 
Dok genre.indie ima smisla ukloniti jer on zapravo predstavlja samo da li je kompanija koja je proizvela ovo nezavisna i uobičajeno sa malim budžetom, ostali faktori fps, survival i simulation dosta zavise od skupa podataka koje uzmemo. Menjanjem veličine skupa podataka znatno se menjaju koeficijenti za ove faktore. Zbog ovog razloga, ostavićemo ih i dalje u modelu. 

Faktori koji dosta utiču na broj igrača jesu: 
- company budget -> Naš model govori da što je budžet veći, to ima manje igrača što deluje apsurdno. Međutim, ovo može ukazivati da je trailer views u relaciji sa budžetom kompanije i ukoliko je budžet veliki, ništa ne znači ako video igra nema dovoljno prikaza. 
- trailer views -> Direktno govori o popularnosti igrice
- adventure -> Adventure zahteva veći budžet, ali se igrači smanjuju. Ovo može ukazivati na zasićenost marketa za adventure žanrom.
- open world -> Broj igrača je znatno veći sa tim da zahteva takođe veći budžet. 

R-squared (uncentered):                   0.669
Adj. R-squared (uncentered):              0.513
RMSE on validation set: 147096.0935324357

Budući da naš dataset sadrži 63 igrice gde su igrice izdeljene na žanrove, modelu je možda teško napraviti šablon. Zbog ovoga, model je isproban bez žanr sekcije. 

In [ ]:
# =====================================================================================================
#                                         coef    std err          t      P>|t|      [0.025      0.975]
# -----------------------------------------------------------------------------------------------------
# year_of_release                      31.6192     25.813      1.225      0.227     -20.404      83.643
# company_budget                    -3.382e-05      0.000     -0.147      0.883      -0.000       0.000
# trailer_views                         0.0039      0.002      2.395      0.021       0.001       0.007
# multiplayer                        1.992e+04   4.27e+04      0.466      0.644   -6.62e+04    1.06e+05
# platform_availability.PLAYSTATION  -5.33e+04   8.05e+04     -0.662      0.512   -2.16e+05    1.09e+05
# platform_availability.XBOX          5.23e+04   7.45e+04      0.702      0.486   -9.79e+04    2.02e+05
# =====================================================================================================

Ovaj model prikazuje da budžet nije toliko bitan već prikazi na trejleru.

R-squared (uncentered):                   0.424
Adj. R-squared (uncentered):              0.346
RMSE on validation set: 84506.99398076316

Iako model bez žanrova ima manji R-squared od modela sa žanrovima, RMSE je znatno manji. Zbog ovog razloga, uzećemo model kao glavni.  

#### Linearne pretpostavke

In [ ]:
# ==============================================================================
# Durbin-Watson 1.429 -> Vrednost oko 2 ukazuje na odsustvo autokorelacije 
# ==============================================================================

In [ ]:
# GOLDFELD-QUANDT TEST ZA JEDNAKU VARIJANSU
gq_test = sms.het_goldfeldquandt(results.resid, X_train)
print("\nGoldfeld-Quandt test:")
print("Test statistic:", gq_test[0])
print("p-value:", gq_test[1])

# PRINT
# Goldfeld-Quandt test:
# Test statistic: 1.5497611372004256
# p-value: 0.1739512524284861

Vrednost iznad 0.05 ukazuje na jednaku varijansu.

In [ ]:
train_predictions = model.predict(X_val)
plt.scatter(y_val, train_predictions)
plt.xlabel("Stvarne vrednosti")
plt.ylabel("Predviđene vrednosti")
plt.title("Scatter plot: Stvarne vs. Predviđene vrednosti")
plt.show()

![Linearnost](linearity.png)

Linearne pretpostavke su donekle ispoštovane. 

## Korišćenje i GUI

Da bi se pokrenuo program, treba pokrenuti main. 

In [ ]:
if __name__ == "__main__":
    # test_model(1, RegressionModel.OLS)
    app = QApplication(sys.argv)
    game_prediction_window = DisclaimerWindow()
    game_prediction_window.show()
    sys.exit(app.exec_())

- Ukoliko neko želi da testira model, otkomentarisati liniju test_model(1, RegressionModel.OLS). 
- Ukoliko se prvi parametar stavi na bilo šta drugo od 1, koristiće se players_after_1year kao ciljnu promenljivu umesto players_on_release.
- Ukoliko neko želi da testira druge modele, promeniti 2. parametar na željeni model. 

GUI koristi model bez žanrova. 
Međutim, žanrovi jesu dostupni ukoliko će se dataset i model menjati u budućnosti. 

Aplikacija procenjuje da će GTA VI imati 800_000 (+- 80000) igrača na Steam platformi, što deluje razumno budući da je GTA V imao 350_000 u trenutku izlaska. 

## Zaključak

Model koji se izrađivao u ovom projektu je doneo umereno dobre rezultate uzimajući u obzir veličinu dataset-a koji mu je bio dostupan. 
Ukoliko se nađe način da se automatski nabavljaju statistike sa pouzdanog i konstantnog izvora, model bi definitivno imao veći potencijal. 